In [ ]:
import glob
import json
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
import pickle

#### Load simpleQA dataset 

In [ ]:
! ls ../uopenie_qa/SimpleWikidataQuestions/csv\ decoded/

In [ ]:
import pandas as pd

data = []

for file in [
#     "../uopenie_qa/SimpleWikidataQuestions/csv decoded/annotated_wd_data_test_answerable_decoded.csv",
#     "../uopenie_qa/SimpleWikidataQuestions/csv decoded/annotated_wd_data_train_answerable_decoded.csv",
    "../uopenie_qa/SimpleWikidataQuestions/csv decoded/annotated_wd_data_valid_answerable_decoded.csv",
]:
    data.append(pd.read_csv(file))
    
data = pd.concat(data)    
data.subject_decoded = data.subject_decoded.map(lambda row: row.strip())
data.object_decoded = data.object_decoded.map(lambda row: row.strip())
data.property_decoded = data.property_decoded.map(lambda row: row.strip())
data.head(20)

In [ ]:
mapper = {}

for idx in data.index:
    if not mapper.get(data.iloc[idx]['property_decoded']):
        mapper[data.iloc[idx]['property_decoded']] = data.iloc[idx]['mapped_rel']

In [ ]:
mapper

In [ ]:
data.shape

In [ ]:
# use for mapper initialization on test data

# mapper = {}

# for property in data.property_decoded.unique():
#     print(property, flush=True)
#     mapper[property] = input()

In [ ]:
data['mapped_rel'] = data.property_decoded.map(lambda row: mapper.get(row) or row)

In [ ]:
data.to_csv("../uopenie_qa/SimpleWikidataQuestions/csv decoded/annotated_wd_data_valid_answerable_decoded.csv")

In [ ]:
import pickle
from tqdm import tqdm_notebook as tqdm
import glob
import os


trex_path = 'trex_data'
aligned_trex_path = 'trex_simpleqa_aligned'

! mkdir $aligned_trex_path 

#clean_dataset = {}  #pickle.load(open(f'{aligned_trex_path}.pkl', 'rb'))
counter = 0

for dataset_file in tqdm(glob.glob(os.path.join(trex_path, '*.json'))[139:156]):
    dataset = pd.read_json(dataset_file)
    clean_rows = []
    
    for document in tqdm(range(dataset.shape[0])):
#     for document in tqdm(range(100)):
        entity = dataset.iloc[document].docid.split('/')[-1]
        for arguments in data[['subject_encoded', 'object_encoded']].values:
            if entity in arguments:
                clean_rows.append(document)
                break
                
    clean_dataset[dataset_file] = clean_rows
    counter += 1
    
    if counter == 5:
        pickle.dump(clean_dataset, open(f'{aligned_trex_path}.pkl', 'wb'))
        counter = 0

In [ ]:
pickle.dump(clean_dataset, open(f'{aligned_trex_path}.pkl', 'wb'))